# 102Flowers Image Classifier

This is the main notebook for the project. See the associated report (WIP) for more information.

This document trains the model and gives an accuracy of roughly 15% after 100 epochs.
The problem with this is that it is overfitting to the training set and therefore it will not predict correctly for a new dataset of flowers. Therefore there is a a very low accuracy using the verification training set.

### Imports

In [373]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets, transforms, models

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

### Hyperparameters

In [374]:
# TODO: Set hyperparameters.
training_batch_size = 20
validation_batch_size = 20
test_batch_size = 20
epochs = 30
learning_rate = 0.001
#momentum = 0.9
crop_size = 128

### Device

In [375]:
# Default to CPU
device = torch.device("cpu")

# Switch to GPU if available
if torch.cuda.is_available():
	print(f"Found {torch.cuda.device_count()} GPUs. Using cuda:0.")
	device = torch.device("cuda:0")
else:
	print("No GPUs found, using CPU.")

Found 1 GPUs. Using cuda:0.


### Load Dataset

In [376]:
training_data = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

validation_data = datasets.Flowers102(
    root = "data",
    split = "val",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

testing_data = datasets.Flowers102(
    root = "data",
    split = "test",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

### DataLoaders

In [377]:
train_dataloader = DataLoader(training_data, batch_size=training_batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=validation_batch_size, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=test_batch_size, shuffle=True)

## Model

In [378]:
classifications = F.one_hot(torch.tensor([e for e in range(0,102)], device="cuda:0"), num_classes=102)
classifications, classifications.size()

(tensor([[1, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 0, 1,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 1, 0, 0],
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 0, 0, 1]], device='cuda:0'),
 torch.Size([102, 102]))

In [379]:
class F102Classifier(nn.Module):
    
	def __init__(self):
		super(F102Classifier, self).__init__()
		
		self.pool = nn.MaxPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.conv2 = nn.Conv2d(6, 12, 3)
		self.conv3 = nn.Conv2d(12, 24, 3)
		self.conv4 = nn.Conv2d(24, 48, 3)
		self.conv5 = nn.Conv2d(48, 96, 3)
		self.fc1 = nn.Linear(384, 102)

	def forward(self, x):
		x = self.pool(F.relu(self.conv1(x)))
		x = self.pool(F.relu(self.conv2(x)))
		x = self.pool(F.relu(self.conv3(x)))
		x = self.pool(F.relu(self.conv4(x)))
		x = self.pool(F.relu(self.conv5(x)))
		#print("after conv1: ", x.size())
		
		x = torch.flatten(x)
		#print("after flatten: ", x.size())
		x = x.view(training_batch_size, -1)
		#print("after view: ", x.size())
		#x = F.relu(self.fc1(x))
		x = self.fc1(x)
		#print("after fc3: ", x.size())
		#print(x)
		#print(x.size())
		return x

net = F102Classifier()
if device == torch.device("cuda:0"):
	net.cuda()

### Loss Function & Optimiser

In [380]:
loss_function = nn.CrossEntropyLoss()
optimiser = optim.Adam(net.parameters(), lr=learning_rate)

## Train

In [381]:
def train(dataloader, model, loss_fn, optimizer):
	epoch = 1
	for batch, (i,j) in enumerate(dataloader):
		features, labels = i.to(device), j.to(device)
  
		# Compute the loss based off the predictions vs labels
		predictions = model(features)
		loss = loss_fn(predictions, labels)

		#Compute back propagation
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()

		if (batch+1) % 51 == 0:
			print(f'Average Loss in Epoch: {loss.item()}')
print('Finished Training')

Finished Training


### Testing

In [382]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.cuda()
    model.eval()
    test_loss, correct = 0, 0
    for batch, (i,j) in enumerate(dataloader):
        features, labels = i.to(device), j.to(device)
        model.cuda()
        pred = model(features)
        test_loss += loss_fn(pred, labels).item()
        correct += (pred.argmax(1) == labels).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss : {test_loss:>8f} \n")

### Running Training and Testing

In [383]:
for t in range(epochs):
    print(f'Epoch {t+1}-------------')
    train(train_dataloader, net, loss_function, optimiser)

test(validation_dataloader, net, loss_function)

Epoch 1-------------


RuntimeError: mat1 and mat2 shapes cannot be multiplied (20x86400 and 384x102)

### Save Model

In [ ]:
""" 30,0.001 = 15.2%
 100, 0.001 = 14.3%
 50, 0.001 = 16.4%
 30,0.01 = 1.0%  Herma-OF"""

"""For Theo	

        self.pool = nn.AvgPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.conv2 = nn.Conv2d(6, 12, 3)
		self.conv3 = nn.Conv2d(12, 24, 3)
		self.conv4 = nn.Conv2d(24, 48, 3)
		self.conv5 = nn.Conv2d(48, 96, 3)
		self.fc1 = nn.Linear(384, 1024)
		self.fc2 = nn.Linear(1024, 512)
		self.fc3 = nn.Linear(512, 102)	
30, 0.01 = 1.0%
30, 0.001 = 14.0%
100, 0.001 = 13.2%
30, 0.0001 = 7.2%

		self.pool = nn.AvgPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 12, 3)
		self.conv2 = nn.Conv2d(12, 48, 3)
		self.conv3 = nn.Conv2d(48, 96, 3)
		self.fc1 = nn.Linear(18816, 1024)
		self.fc2 = nn.Linear(1024, 512)
		self.fc3 = nn.Linear(512, 102)		
100, 0.001 = 9.5%
30, 0.001 = 17.8%
50, 0.001 = 1.0%

		self.pool = nn.AvgPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 12, 3)
		self.fc1 = nn.Linear(47628, 102)
30, 0.001 = 19.9%

		self.pool = nn.AvgPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.fc1 = nn.Linear(23814, 102)
30, 0.001 = 17.9%
15, 0.001 = 16.0%
10, 0.001 = 19.0%
5, 0.001 = 14.4%

		self.pool = nn.MaxPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.fc1 = nn.Linear(23814, 102)
30, 0.001 = 19.2%
10, 0.001 = 16.8%
50, 0.001 = 17.0

		self.pool = nn.MaxPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.conv2 = nn.Conv2d(6, 12, 3)
		self.conv3 = nn.Conv2d(12, 24, 3)
		self.conv4 = nn.Conv2d(24, 48, 3)
		self.conv5 = nn.Conv2d(48, 96, 3)
		self.fc1 = nn.Linear(384, 102)
30, 0.001 = 18.6%
 """
save_path = "./models/classifier.pth"
torch.save(net.state_dict(), save_path)